In [1]:
# Install packages
!pip install google-cloud-aiplatform PyPDF2 functions-framework --quiet

# Authenticate Colab
from google.colab import auth
auth.authenticate_user()

# Set credentials
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gemini-key.json"

print("Setup complete!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
Setup complete!


In [2]:
# main.py – RAG API with Gemini 2.0 Flash

import vertexai
from vertexai.generative_models import GenerativeModel
import PyPDF2
import functions_framework

# Initialize Vertex AI
vertexai.init(project="bamboo-theorem-2000", location="us-central1")
model = GenerativeModel("gemini-2.0-flash")

# Load PDF
def load_pdf(file_path):
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

pdf_text = load_pdf("AI_Prompt_Engineer_Role.pdf")

# RAG API endpoint
@functions_framework.http
def rag_api(request):
    data = request.get_json(silent=True) or {}
    question = data.get("question", "What does a Prompt Engineer do?")
    prompt = f"Document: {pdf_text}\nQuestion: {question}\nAnswer (short):"
    response = model.generate_content(prompt)
    return {"answer": response.text}

/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [3]:
# requirements.txt
with open("requirements.txt", "w") as f:
    f.write("""google-cloud-aiplatform>=1.38
PyPDF2
functions-framework
""")
print("requirements.txt created!")

requirements.txt created!


In [4]:
# Simulate HTTP request
class FakeRequest:
    def __init__(self, json_data):
        self._json = json_data
    def get_json(self, silent=False):
        return self._json

# Test
test_data = {"question": "What are the main tasks of an AI Prompt Engineer?"}
result = rag_api(FakeRequest(test_data))

print("LOCAL TEST RESULT:")
print(result["answer"])

LOCAL TEST RESULT:
Craft prompts, test AI outputs, collaborate on integration, and use GCP.



In [12]:
# main.py – RAG API (English, clean)
with open("main.py", "w") as f:
    f.write('''\
import vertexai
from vertexai.generative_models import GenerativeModel
import PyPDF2
import functions_framework

vertexai.init(project="bamboo-theorem-2000", location="us-central1")
model = GenerativeModel("gemini-2.0-flash")

def load_pdf(file_path):
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\\n"
    return text

pdf_text = load_pdf("AI_Prompt_Engineer_Role.pdf")

@functions_framework.http
def rag_api(request):
    data = request.get_json(silent=True) or {}
    question = data.get("question", "What does a Prompt Engineer do?")
    prompt = f"Document: \\"{pdf_text}\\" Question: {question} Answer:"
    response = model.generate_content(prompt)
    return {"answer": response.text}
''')

# requirements.txt
with open("requirements.txt", "w") as f:
    f.write("""\
google-cloud-aiplatform>=1.38
PyPDF2
functions-framework
""")

print("main.py + requirements.txt created!")

main.py + requirements.txt created!


In [13]:
!gcloud config set project bamboo-theorem-2000
!gcloud config get-value project

Updated property [core/project].
bamboo-theorem-2000


In [14]:
!gcloud run deploy rag-api \
  --source . \
  --platform managed \
  --region us-central1 \
  --allow-unauthenticated \
  --function rag_api \
  --memory 256Mi \
  --timeout 60s \
  --project bamboo-theorem-2000 \
  --quiet

Building using Buildpacks and deploying container to Cloud Run service [rag-api] in project [bamboo-theorem-2000] region [us-central1]
Service [rag-api] revision [rag-api-00002-qll] has been deployed and is serving 100 percent of traffic.
Service URL: https://rag-api-116522765677.us-central1.run.app


In [15]:
import requests

# ← ZMEŇ NA SVOJU URL!
url = "https://rag-api-116522765677.us-central1.run.app"

data = {"question": "What are the main tasks of an AI Prompt Engineer?"}

response = requests.post(url, json=data)

print("LIVE API RESPONSE:")
print(response.json()["answer"])

LIVE API RESPONSE:
Crafting effective prompts, testing and improving AI outputs, collaborating with developers on AI integration, and using GCP for deploying solutions.

